### 人类活动识别

In [ ]:
# 加载类别信息

from utils.ConfigUtils import get_classes

classes=list(get_classes(r"data\RFID_multi_628\data.yml").values())
num_classes = len(classes)

print(classes)

In [ ]:
# 数据预处理
from torchvision import transforms
from model.Normalization import *

transform=transforms.Compose([
    RobustNorm(-68.0, 68.0),
])

In [ ]:
# 加载数据集
from model.RFID_Dataset import RFID_Dataset

train_dir = r"data\RFID_multi_628\dataset\train"
eval_dir = r"data\RFID_multi_628\dataset\eval"
generate_dir="./output/base"

train_dataset = RFID_Dataset(
    train_dir,
    T=32,
    step=1,
    num_channels=3,
    transform=transform,
)

eval_dataset = RFID_Dataset(
    eval_dir,
    T=32,
    step=1,
    num_channels=3,
    transform=transform,
)

generate_dataset = RFID_Dataset(
    generate_dir,
    T=32,
    step=32,
    num_channels=3,
    transform=transforms.Compose([
    # RobustNorm(-1.0, 1.0),
]),
)

# train_dataset=generate_dataset

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 加载数据集
from model.RFID_Dataset import RFID_Dataset
from torch.utils.data import ConcatDataset

dir_group = [
    r"data\RFID_multi_628\dataset\person\hyx",
    r"data\RFID_multi_628\dataset\person\lrf",
    r"data\RFID_multi_628\dataset\person\ljl",
    r"data\RFID_multi_628\dataset\person\xjy",
    r"data\RFID_multi_628\dataset\person\hjx",
]

dataset_group=[]
for dir in dir_group:
    dataset_group.append(RFID_Dataset(
        dir,
        T=32,
        step=1,
        num_channels=3,
        transform=transform,
    ))

person_count=3
train_dataset=ConcatDataset(dataset_group[:person_count])
eval_dataset=ConcatDataset(dataset_group[person_count:])

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网
# from model.ClassifyNet.MixedMemoryNet import MixedMemoryNet as ClassifyNet
from model.ClassifyNet.CNNClassifyNet import CNNClassifyNet as ClassifyNet
from model.ModelWorker.ClassifyModelWorker import ClassifyModelWorker
from torchkeras import summary

input_shape=(3,32,12)
model=ClassifyNet(
    input_shape=input_shape,
    num_classes=num_classes
)

model_worker=ClassifyModelWorker(model)

print(f"{input_shape=}")
model_info=summary(model,input_shape=input_shape)

In [ ]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torchmetrics import Accuracy,F1Score
from model.LightningModel.ClassifyPLModel import ClassifyPLModel

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32)

loss = nn.CrossEntropyLoss()
metrics= {
    "F1Score":F1Score(task="multiclass", num_classes=num_classes)
}

pl_model = ClassifyPLModel(
    model,
    loss,
    metrics
)

In [ ]:
# 构建PLTrainer
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping

trainer = pl.Trainer(
    max_epochs=100,
    min_epochs=5,
    logger=True,
    callbacks=[
        ModelCheckpoint(save_weights_only=True),
        # EarlyStopping(monitor="val/loss", patience=5),
        EarlyStopping(monitor="val/F1Score", patience=5,mode="max"),
    ],
    default_root_dir="./output/HAR",
)

In [ ]:
# 模型训练

trainer.fit(
    pl_model,
    train_loader,
    eval_loader,
)
best_model_path=trainer.checkpoint_callback.best_model_path
print(best_model_path)

In [ ]:
# 加载PLModel

best_model_path=r"output\HAR\lightning_logs\version_7\checkpoints\epoch=99-step=6100.ckpt"
pl_model=ClassifyPLModel.load_from_checkpoint(
    best_model_path, 
    model=model, 
    criterion=loss,
    metrics=metrics,
)

In [ ]:
test_dir="./output/base"
test_dataset=eval_dataset = RFID_Dataset(
    test_dir,
    T=32,
    step=32,
    num_channels=3,
    # transform=transform,
)
test_loader = DataLoader(test_dataset, batch_size=4)

print(f"测试集的数据个数: {len(test_dataset)}")


In [ ]:
# 模型评估

trainer.validate(
    pl_model,
    test_loader,
)

In [ ]:
# 评价指标

from torchmetrics import MetricCollection, Accuracy, F1Score,ConfusionMatrix

used_dataset = "test_dataset"
dataset_group={
    "train_dataset": train_loader,
    "eval_dataset": eval_loader,
    "test_dataset": test_loader,
}
res=model_worker.execute_metric(
    dataset_group[used_dataset],
    MetricCollection([
        # Accuracy(task="multiclass", num_classes=num_classes),
        F1Score(task="multiclass", num_classes=num_classes),
        ConfusionMatrix(task="multiclass", num_classes=num_classes,normalize="true")
    ]),
)


In [ ]:
# 展示混淆矩阵

from utils.DataUtils.Visualization import plot_confusion_matrix

title=f"{used_dataset}, F1Score: {res['MulticlassF1Score']:.6f}"
confusion_matrix=res["MulticlassConfusionMatrix"].cpu()
plot_confusion_matrix(
    confusion_matrix,
    class_names=classes,
    is_percentage=True,
    title=title,
)



In [ ]:
# 保存模型
model_worker.save('./output/HAR/HAR.pth')

In [ ]:
# 加载模型
model_worker.load('./output/HAR/HAR.pth')

In [ ]:
# 模型预测
from utils.DatasetUtils import DatasetUtils

used_dataset = eval_dataset
indexs,inputs,labels = DatasetUtils().select_simple(used_dataset,count=4)
preds,acc = model_worker.predict(inputs=inputs,labels=labels)

print(f"{inputs.shape=}")
print(f"{acc=}")
print(f"indexs:{indexs}")
print(f"labels:{labels}")
print(f"preds :{preds}")